In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_hugectr_hps-hierarchical-parameter-server-demo/nvidia_logo.png" style="width: 90px; float: right;">

# HPS Torch Demo

## Overview

Hierarchical Parameter Server (HPS) is a distributed recommendation inference framework, which combines a high-performance GPU embedding cache with an hierarchical storage architecture, to realize low-latency retrieval of embeddings for inference tasks. It is provided as a PyTorch plugin and can be easily used in the Torch model.

This notebook demonstrates how to apply HPS to the Torch model and use it for inference. For more details about HPS APIs, please refer to [HPS APIs](https://nvidia-merlin.github.io/HugeCTR/main/hierarchical_parameter_server/hps_torch_api/index.html). For more details about HPS, please refer to [HugeCTR Hierarchical Parameter Server (HPS)](https://nvidia-merlin.github.io/HugeCTR/main/hierarchical_parameter_server/index.html).

## Installation

### Get HPS from NGC

The HPS Python module is preinstalled in the 23.12 and later [Merlin PyTorch Container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-pytorch): `nvcr.io/nvidia/merlin/merlin-pytorch:23.12`.

You can check the existence of the required libraries by running the following Python code after launching this container.

```bash
$ python3 -c "import hps_torch"
```

## Data Generation

First of all we specify the required configurations for data generation. We generate 8 embedding tables, all with the same embedding vector size 128. The maximum batch size is 256 and each sample has 10 keys to lookup up for each table.

In [1]:
import torch
import hps_torch
from typing import List
import os
import numpy as np
import struct
import json
import pytest
import time

NUM_GPUS = 1
VOCAB_SIZE = 10000
EMB_VEC_SIZE = 128
NUM_QUERY_KEY = 10
MAX_BATCH_SIZE = 256
NUM_ITERS = 100
NUM_TABLES = 8
USE_CONTEXT_STREAM = True

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, range(NUM_GPUS)))

[INFO] hps_torch is imported


In [2]:
hps_config = {
    "supportlonglong": False,
    "fuse_embedding_table": True,
    "models": [
        {
            "model": str(NUM_TABLES) + "_table",
            "sparse_files": [],
            "num_of_worker_buffer_in_pool": NUM_TABLES,
            "embedding_table_names": [],
            "embedding_vecsize_per_table": [],
            "maxnum_catfeature_query_per_table_per_sample": [],
            "default_value_for_each_table": [0.0],
            "deployed_device_list": [0],
            "max_batch_size": MAX_BATCH_SIZE,
            "cache_refresh_percentage_per_iteration": 1.0,
            "hit_rate_threshold": 1.0,
            "gpucacheper": 1.0,
            "gpucache": True,
            "embedding_cache_type": "static",
            "use_context_stream": True,
        }
    ],
}

def generate_embedding_tables(
    hugectr_sparse_model, vocab_range, embedding_vec_size, embedding_table
):
    os.system("mkdir -p {}".format(hugectr_sparse_model))
    with open("{}/key".format(hugectr_sparse_model), "wb") as key_file, open(
        "{}/emb_vector".format(hugectr_sparse_model), "wb"
    ) as vec_file:
        for key in range(vocab_range[0], vocab_range[1]):
            vec = np.random.random((embedding_vec_size,)).astype(np.float32)
            key_struct = struct.pack("q", key)
            vec_struct = struct.pack(str(embedding_vec_size) + "f", *vec)
            key_file.write(key_struct)
            vec_file.write(vec_struct)
            embedding_table[key] = vec


def set_up_model_files():
    embedding_table = np.zeros((NUM_TABLES * VOCAB_SIZE, EMB_VEC_SIZE)).astype(np.float32)
    for i in range(NUM_TABLES):
        table_name = "table" + str(i)
        model_file_name = "embeddings/" + table_name
        generate_embedding_tables(
            model_file_name, [i * VOCAB_SIZE, (i + 1) * VOCAB_SIZE], EMB_VEC_SIZE, embedding_table
        )
        hps_config["models"][0]["sparse_files"].append(model_file_name)
        hps_config["models"][0]["embedding_table_names"].append(table_name)
        hps_config["models"][0]["embedding_vecsize_per_table"].append(EMB_VEC_SIZE)
        hps_config["models"][0]["maxnum_catfeature_query_per_table_per_sample"].append(
            NUM_QUERY_KEY
        )
    hps_config_json_object = json.dumps(hps_config, indent=4)
    with open(str(NUM_TABLES) + "_table.json", "w") as outfile:
        outfile.write(hps_config_json_object)
    return embedding_table

In [3]:
embedding_table = set_up_model_files()

In [4]:
!du -lh embeddings

5.0M	embeddings/table0
5.0M	embeddings/table1
5.0M	embeddings/table2
5.0M	embeddings/table3
5.0M	embeddings/table4
5.0M	embeddings/table5
5.0M	embeddings/table6
5.0M	embeddings/table7
40M	embeddings


## Lookup with Table Fusion

HPS supports fusing tables of the same embedding vector size via CPU multithreading. This can be achieved with `torch.jit.fork` and `torch.jit.wait` when the HPS plugin for Torch is employed. For more details, please refer to [HPS Configuration](https://nvidia-merlin.github.io/HugeCTR/main/hierarchical_parameter_server/hps_database_backend.html#configuration).

We conduct embedding lookup with table fusion and compare the results with the ground truth. 

In [5]:
class Model(torch.nn.Module):
    def __init__(self, ps_config_file: str, model_name: str, emb_vec_size: List[int]):
        super().__init__()
        self.layers = torch.nn.ModuleList(
            [
                hps_torch.LookupLayer(ps_config_file, model_name, table_id, emb_vec_size[table_id])
                for table_id in range(len(emb_vec_size))
            ]
        )

    def forward(self, keys_list: torch.Tensor):
        vectors = []
        futures = torch.jit.annotate(List[torch.jit.Future[torch.Tensor]], [])
        for i, layer in enumerate(self.layers):
            fut = torch.jit.fork(layer, keys_list[i])
            futures.append(fut)
        for i, _ in enumerate(self.layers):
            vectors.append(torch.jit.wait(futures[i]))
        return torch.cat(vectors)

In [6]:
model = torch.jit.script(
    Model(
        f"{NUM_TABLES}_table.json",
        f"{NUM_TABLES}_table",
        [EMB_VEC_SIZE for _ in range(NUM_TABLES)],
    )
)
inputs_seq = []
for _ in range(NUM_ITERS + 1):
    inputs = []
    for i in range(NUM_TABLES):
        inputs.append(
            torch.randint(
                i * VOCAB_SIZE,
                (i + 1) * VOCAB_SIZE,
                (MAX_BATCH_SIZE, NUM_QUERY_KEY),
                dtype=torch.int32,
            ).cuda()
        )
    inputs_seq.append(torch.stack(inputs))

preds = model(inputs_seq[0])
preds_seq = []
start = time.time()
for i in range(NUM_ITERS):
    preds_seq.append(model(inputs_seq[i + 1]))
end = time.time()
print(
    "[INFO] Elapsed time for "
    + str(NUM_ITERS)
    + " iterations: "
    + str(end - start)
    + " seconds"
)
preds_seq = torch.stack(preds_seq).cpu().numpy()

preds_seq_gt = []
for i in range(NUM_ITERS):
    preds_seq_gt.append(np.concatenate(embedding_table[inputs_seq[i + 1].cpu().numpy()]))
preds_seq_gt = np.array(preds_seq_gt)

diff = preds_seq - preds_seq_gt
mse = np.mean(diff * diff)
assert mse <= 1e-6
print(f"HPS Torch Plugin embedding lookup with table fusion, MSE: {mse} ")

=====================================================HPS Parse====================================================
[HCTR][05:25:11.836][INFO][RK0][main]: Table fusion is enabled for HPS. Please ensure that there is no key value overlap in different tables and the embedding lookup layer has no dependency in the model graph. For more information, see https://nvidia-merlin.github.io/HugeCTR/main/hierarchical_parameter_server/hps_database_backend.html#configuration
[HCTR][05:25:11.836][INFO][RK0][main]: fuse_embedding_table is not specified using default: 1
[HCTR][05:25:11.839][INFO][RK0][main]: dense_file is not specified using default: 
[HCTR][05:25:11.839][WARNING][RK0][main]: default_value_for_each_table.size() is not equal to the number of embedding tables
[HCTR][05:25:11.839][INFO][RK0][main]: num_of_refresher_buffer_in_pool is not specified using default: 1
[HCTR][05:25:11.839][INFO][RK0][main]: maxnum_des_feature_per_sample is not specified using default: 26
[HCTR][05:25:11.839][IN